<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/data/Spotify_Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

rs_df = pd.read_csv('rs.csv')
rs_df.head()

,2003 #,2012 #,2020 #,Artist,Album,Year,Net Shift (2012-2020)
0,6,6.0,1.0,"Gaye, Marvin",What's Going On,1971,5.0
1,2,2.0,2.0,Beach Boys,Pet Sounds,1966,0.0
2,30,30.0,3.0,"Mitchell, Joni",Blue,1971,27.0
3,57,57.0,4.0,"Wonder, Stevie",Songs in the Key of Life,1976,53.0
4,14,14.0,5.0,Beatles,Abbey Road,1969,9.0


In [6]:
rs_2023_df = pd.read_excel('rs_2023.xlsx')
rs_2023_df.head()

,2023 #,Album,Artist(s)
0,1,What's Going On,Marvin Gaye
1,2,Pet Sounds,The Beach Boys
2,3,Blue,Joni Mitchell
3,4,Songs in the Key of Life,Stevie Wonder
4,5,Abbey Road,The Beatles


In [7]:
import pandas as pd

# Assuming your original DataFrame is named 'rs_df' and the 2023 DataFrame is named 'rs_2023_df'

# Rename the '2023 #' column to 'rank_2023' in rs_2023_df
rs_2023_df = rs_2023_df.rename(columns={'2023 #': 'rank_2023'})

# Merge the original DataFrame with the 2023 DataFrame based on the 'Album' column
merged_df = pd.merge(rs_df, rs_2023_df[['Album', 'Artist(s)', 'rank_2023']], on='Album', how='left')

# Fill missing values in the 'Artist' column with values from 'Artist(s)'
merged_df['Artist'] = merged_df['Artist'].fillna(merged_df['Artist(s)'])

# Drop the 'Artist(s)' and 'Net Shift (2012-2020)' columns
merged_df = merged_df.drop(columns=['Artist(s)', 'Net Shift (2012-2020)'])

# Rename the rank columns
merged_df = merged_df.rename(columns={
    '2003 #': 'rank_2003',
    '2012 #': 'rank_2012',
    '2020 #': 'rank_2020'
})

# Identify the unmatched albums from 2023
unmatched_albums = rs_2023_df[~rs_2023_df['Album'].isin(rs_df['Album'])]

# Concatenate the unmatched albums to the merged DataFrame
final_df = pd.concat([merged_df, unmatched_albums[['Album', 'Artist(s)', 'rank_2023']]], ignore_index=True)

# Fill missing values in the 'Artist' column with values from 'Artist(s)' for unmatched albums
final_df['Artist'] = final_df['Artist'].fillna(final_df['Artist(s)'])

# Drop the 'Artist(s)' column
final_df = final_df.drop(columns=['Artist(s)'])

# Print the resulting DataFrame
final_df

,rank_2003,rank_2012,rank_2020,Artist,Album,Year,rank_2023
0,6,6.0,1.0,"Gaye, Marvin",What's Going On,1971.0,1.0
1,2,2.0,2.0,Beach Boys,Pet Sounds,1966.0,2.0
2,30,30.0,3.0,"Mitchell, Joni",Blue,1971.0,3.0
3,57,57.0,4.0,"Wonder, Stevie",Songs in the Key of Life,1976.0,4.0
4,14,14.0,5.0,Beatles,Abbey Road,1969.0,NaN
...,...,...,...,...,...,...,...
880,NaN,NaN,NaN,Muddy Waters,The Anthology: 1947–1972,NaN,483.0
881,NaN,NaN,NaN,Lady Gaga,Born This Way,NaN,484.0
882,NaN,NaN,NaN,Various artists,Back to Mono (1958–1969),NaN,489.0
883,NaN,NaN,NaN,Harry Styles,Harry's House,NaN,491.0


In [8]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 5.8 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotify API credentials
client_id = 'c9709c57231f4652b9eaf718525d52ad'
client_secret = '74a1b4e6ae23440b85acc0334e9b6ffb'

# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to search for an album on Spotify and retrieve additional information
def get_album_info(album_name, artist_name):
    results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album', limit=1)
    if results['albums']['items']:
        album = results['albums']['items'][0]
        album_id = album['id']
        album_url = album['external_urls']['spotify']
        release_date = album['release_date']
        total_tracks = album['total_tracks']
        return album_id, album_url, release_date, total_tracks
    else:
        return None, None, None, None

# Iterate over the albums in your DataFrame and retrieve additional information
final_df['spotify_id'], final_df['spotify_url'], final_df['release_date'], final_df['total_tracks'] = zip(*final_df.apply(lambda row: get_album_info(row['Album'], row['Artist']), axis=1))

# Print the updated DataFrame
final_df

In [10]:
# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to search for an artist on Spotify and retrieve information
def get_artist_info(artist_name):
    results = sp.search(q=f'artist:{artist_name}', type='artist', limit=1)
    if results['artists']['items']:
        artist = results['artists']['items'][0]
        artist_id = artist['id']
        artist_url = artist['external_urls']['spotify']
        followers = artist['followers']['total']
        genres = artist['genres']
        popularity = artist['popularity']
        return artist_id, artist_url, followers, genres, popularity
    else:
        return None, None, None, None, None

# Iterate over the artists in your DataFrame and retrieve information
final_df['artist_id'], final_df['artist_url'], final_df['followers'], final_df['genres'], final_df['popularity'] = zip(*final_df['Artist'].apply(get_artist_info))

# Print the updated DataFrame
print(final_df)

    rank_2003  rank_2012  rank_2020           Artist  \
0           6        6.0        1.0     Gaye, Marvin   
1           2        2.0        2.0       Beach Boys   
2          30       30.0        3.0   Mitchell, Joni   
3          57       57.0        4.0   Wonder, Stevie   
4          14       14.0        5.0          Beatles   
..        ...        ...        ...              ...   
880       NaN        NaN        NaN     Muddy Waters   
881       NaN        NaN        NaN        Lady Gaga   
882       NaN        NaN        NaN  Various artists   
883       NaN        NaN        NaN     Harry Styles   
884       NaN        NaN        NaN          Shakira   

                           Album    Year  rank_2023              spotify_id  \
0                What's Going On  1971.0        1.0  7qD9oSiS3xFJsw2oSV2akp   
1                     Pet Sounds  1966.0        2.0  2CNEkSE8TADXRT2AzcEt1b   
2                           Blue  1971.0        3.0  1vz94WpXDVYIEGja8cjFNa   
3       Son

In [11]:
final_df

,rank_2003,rank_2012,rank_2020,Artist,Album,Year,rank_2023,spotify_id,spotify_url,release_date,total_tracks,artist_id,artist_url,followers,genres,popularity
0,6,6.0,1.0,"Gaye, Marvin",What's Going On,1971.0,1.0,7qD9oSiS3xFJsw2oSV2akp,https://open.spotify.com/album/7qD9oSiS3xFJsw2...,2021-01-22,9.0,3koiLjNrgRTNbOwViDipeA,https://open.spotify.com/artist/3koiLjNrgRTNbO...,5313182.0,"[classic soul, motown, neo soul, northern soul...",71.0
1,2,2.0,2.0,Beach Boys,Pet Sounds,1966.0,2.0,2CNEkSE8TADXRT2AzcEt1b,https://open.spotify.com/album/2CNEkSE8TADXRT2...,1966-05-16,13.0,3oDbviiivRWhXwIE8hxkVV,https://open.spotify.com/artist/3oDbviiivRWhXw...,4499081.0,"[baroque pop, classic rock, folk rock, mellow ...",71.0
2,30,30.0,3.0,"Mitchell, Joni",Blue,1971.0,3.0,1vz94WpXDVYIEGja8cjFNa,https://open.spotify.com/album/1vz94WpXDVYIEGj...,1971-06-22,10.0,5hW4L92KnC6dX9t7tYM4Ve,https://open.spotify.com/artist/5hW4L92KnC6dX9...,1461857.0,"[canadian singer-songwriter, folk, folk rock, ...",59.0
3,57,57.0,4.0,"Wonder, Stevie",Songs in the Key of Life,1976.0,4.0,6YUCc2RiXcEKS9ibuZxjt0,https://open.spotify.com/album/6YUCc2RiXcEKS9i...,1976-09-28,21.0,7guDJrEfX3qb6FEbdPA5qi,https://open.spotify.com/artist/7guDJrEfX3qb6F...,6427524.0,"[motown, soul]",73.0
4,14,14.0,5.0,Beatles,Abbey Road,1969.0,NaN,0ETFjACtuP2ADo6LFhL6HN,https://open.spotify.com/album/0ETFjACtuP2ADo6...,1969-09-26,17.0,3WrFJ7ztbogyGnTHbHJFl2,https://open.spotify.com/artist/3WrFJ7ztbogyGn...,27469908.0,"[british invasion, classic rock, merseybeat, p...",82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,NaN,NaN,NaN,Muddy Waters,The Anthology: 1947–1972,NaN,483.0,None,None,None,NaN,4y6J8jwRAwO4dssiSmN91R,https://open.spotify.com/artist/4y6J8jwRAwO4ds...,1281361.0,"[blues, blues rock, chicago blues, classic roc...",53.0
881,NaN,NaN,NaN,Lady Gaga,Born This Way,NaN,484.0,5maeycU97NHBgwRr2h2A4O,https://open.spotify.com/album/5maeycU97NHBgwR...,2011-01-01,22.0,1HY2Jd0NmPuamShAr6KMms,https://open.spotify.com/artist/1HY2Jd0NmPuamS...,29997737.0,"[art pop, dance pop, pop]",83.0
882,NaN,NaN,NaN,Various artists,Back to Mono (1958–1969),NaN,489.0,None,None,None,NaN,3EokDyGBRqoAQX5LSiEkSU,https://open.spotify.com/artist/3EokDyGBRqoAQX...,37.0,[],4.0
883,NaN,NaN,NaN,Harry Styles,Harry's House,NaN,491.0,6XCpLLxr9adk3jwrHLqRLQ,https://open.spotify.com/album/6XCpLLxr9adk3jw...,2023-03-25,13.0,6KImCVD70vtIoJWnq6nGn3,https://open.spotify.com/artist/6KImCVD70vtIoJ...,32203050.0,[pop],83.0


In [12]:
# Create albums_df
albums_df = final_df[['Album', 'artist_id', 'Year', 'rank_2003', 'rank_2012', 'rank_2020', 'rank_2023', 'spotify_id', 'spotify_url', 'release_date', 'total_tracks']]

# Create artists_df
artists_df = final_df[['Artist', 'artist_id', 'artist_url', 'followers', 'genres', 'popularity']]

# Drop duplicate artist entries
artists_df = artists_df.drop_duplicates(subset='Artist')

# Reset the index of artists_df
artists_df = artists_df.reset_index(drop=True)

In [13]:
artists_df = artists_df[['artist_id', 'Artist', 'followers', 'genres', 'popularity']]
artists_df.head()

,artist_id,Artist,followers,genres,popularity
0,3koiLjNrgRTNbOwViDipeA,"Gaye, Marvin",5313182.0,"[classic soul, motown, neo soul, northern soul...",71.0
1,3oDbviiivRWhXwIE8hxkVV,Beach Boys,4499081.0,"[baroque pop, classic rock, folk rock, mellow ...",71.0
2,5hW4L92KnC6dX9t7tYM4Ve,"Mitchell, Joni",1461857.0,"[canadian singer-songwriter, folk, folk rock, ...",59.0
3,7guDJrEfX3qb6FEbdPA5qi,"Wonder, Stevie",6427524.0,"[motown, soul]",73.0
4,3WrFJ7ztbogyGnTHbHJFl2,Beatles,27469908.0,"[british invasion, classic rock, merseybeat, p...",82.0


In [14]:
albums_df = albums_df[['spotify_id', 'Album', 'artist_id', 'Year', 'rank_2003', 'rank_2012', 'rank_2020', 'rank_2023']]
# rename spotify_id to album_id
albums_df = albums_df.rename(columns={'spotify_id': 'album_id'})


,album_id,Album,artist_id,Year,rank_2003,rank_2012,rank_2020,rank_2023
0,7qD9oSiS3xFJsw2oSV2akp,What's Going On,3koiLjNrgRTNbOwViDipeA,1971.0,6,6.0,1.0,1.0
1,2CNEkSE8TADXRT2AzcEt1b,Pet Sounds,3oDbviiivRWhXwIE8hxkVV,1966.0,2,2.0,2.0,2.0
2,1vz94WpXDVYIEGja8cjFNa,Blue,5hW4L92KnC6dX9t7tYM4Ve,1971.0,30,30.0,3.0,3.0
3,6YUCc2RiXcEKS9ibuZxjt0,Songs in the Key of Life,7guDJrEfX3qb6FEbdPA5qi,1976.0,57,57.0,4.0,4.0
4,0ETFjACtuP2ADo6LFhL6HN,Abbey Road,3WrFJ7ztbogyGnTHbHJFl2,1969.0,14,14.0,5.0,NaN


In [49]:
# drop duplicate album_id
albums_df = albums_df.drop_duplicates(subset='album_id')
# reset index
albums_df = albums_df.reset_index(drop=True)

In [17]:
# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to retrieve tracks of an album from Spotify
def get_album_tracks(album_id):
    tracks = []
    results = sp.album_tracks(album_id)
    for item in results['items']:
        track_id = item['id']
        track_name = item['name']
        track_number = item['track_number']
        track_duration = item['duration_ms']
        track_url = item['external_urls']['spotify']
        tracks.append([track_id, track_name, track_number, track_duration, track_url])
    return tracks

# Create an empty list to store the track data
track_data = []

# Iterate over the albums in albums_df and retrieve track information
for _, row in albums_df.iterrows():
    album_id = row['album_id']
    if pd.notnull(album_id):
        album_tracks = get_album_tracks(album_id)
        for track in album_tracks:
            track_data.append([album_id] + track)

# Create the tracks_df DataFrame
tracks_df = pd.DataFrame(track_data, columns=['album_id', 'track_id', 'track_name', 'track_number', 'duration_ms', 'track_url'])


In [61]:
tracks_df = tracks_df[['track_id',  'track_name', 'album_id', 'track_number', 'duration_ms']]
tracks_df.describe()

,track_number,duration_ms
count,12621.00000,1.262100e+04
mean,8.26860,2.317759e+05
std,6.01361,1.055106e+05
min,1.00000,4.000000e+03
25%,4.00000,1.678660e+05
50%,7.00000,2.179600e+05
75%,11.00000,2.752530e+05
max,50.00000,1.619373e+06


In [62]:
# drop duplicate track ids
tracks_df = tracks_df.drop_duplicates(subset='track_id')
# reset index
tracks_df = tracks_df.reset_index(drop=True)
tracks_df.describe()

,track_number,duration_ms
count,9780.000000,9.780000e+03
mean,8.151431,2.317075e+05
std,5.783431,1.054624e+05
min,1.000000,4.000000e+03
25%,4.000000,1.683165e+05
50%,7.000000,2.173395e+05
75%,11.000000,2.739288e+05
max,50.000000,1.619373e+06


In [64]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('rs_greatest_albums.db')

# Write artists_df to the database
artists_df.to_sql('artists', conn, if_exists='replace', index=False)

# Write albums_df to the database
albums_df.to_sql('albums', conn, if_exists='replace', index=False)

# Write tracks_df to the database
tracks_df.to_sql('tracks', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()

In [39]:
artists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   artist_id   387 non-null    object 
 1   Artist      387 non-null    object 
 2   followers   387 non-null    int64  
 3   genres      387 non-null    object 
 4   popularity  387 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 15.2+ KB


In [35]:
# artist_df
# drop duplicate artist_id rows
artists_df = artists_df.drop_duplicates(subset='artist_id')

# drop rows with null artist_id
artists_df = artists_df.dropna(subset=['artist_id'])

# reset index
artists_df = artists_df.reset_index(drop=True)

In [38]:
# artist_df
# case followers to int
artists_df['followers'] = artists_df['followers'].astype(int)

# cast genres to string
artists_df['genres'] = artists_df['genres'].astype(str)

In [74]:
# connect sql magic
%reload_ext sql
%sql sqlite:///rs_greatest_albums.db

In [75]:
%%sql
SELECT * FROM artists LIMIT 10;

   sqlite:///music_database.db
   sqlite:///music_database_2.db
 * sqlite:///rs_greatest_albums.db
Done.


artist_id,Artist,followers,genres,popularity
3koiLjNrgRTNbOwViDipeA,"Gaye, Marvin",5313182,"['classic soul', 'motown', 'neo soul', 'northern soul', 'quiet storm', 'soul']",71.0
3oDbviiivRWhXwIE8hxkVV,Beach Boys,4499081,"['baroque pop', 'classic rock', 'folk rock', 'mellow gold', 'psychedelic rock', 'rock', 'singer-songwriter', 'sunshine pop']",71.0
5hW4L92KnC6dX9t7tYM4Ve,"Mitchell, Joni",1461857,"['canadian singer-songwriter', 'folk', 'folk rock', 'singer-songwriter']",59.0
7guDJrEfX3qb6FEbdPA5qi,"Wonder, Stevie",6427524,"['motown', 'soul']",73.0
3WrFJ7ztbogyGnTHbHJFl2,Beatles,27469908,"['british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']",82.0
6olE6TJLqED3rqDCT0FyPh,Nirvana,19435759,"['grunge', 'permanent wave', 'rock']",80.0
08GQAI4eElDnROBrJRGE0X,Fleetwood Mac,11482299,"['album rock', 'classic rock', 'rock', 'soft rock', 'yacht rock']",79.0
5a2EaR3hamoenG9rDuVn8j,Prince and the Revolution,7275856,"['funk', 'funk rock', 'minneapolis sound', 'rock', 'synth funk']",69.0
74ASZWbe4lXaubB36ztrGX,"Dylan, Bob",6456033,"['classic rock', 'country rock', 'folk', 'folk rock', 'rock', 'roots rock', 'singer-songwriter']",69.0
2Mu5NfyYm8n5iTomuKAEHl,"Hill, Lauryn",2895532,"['alternative r&b', 'conscious hip hop', 'hip hop', 'neo soul', 'new jersey rap', 'r&b']",71.0


In [76]:
%%sql
-- create new_arists with artist_id as primary key
DROP TABLE IF EXISTS new_artists;
CREATE TABLE new_artists (
    artist_id STRING PRIMARY KEY,
    Artist TEXT,
    followers INTEGER,
    genres TEXT,
    popularity INTEGER
);

-- insert data into new_artists
INSERT INTO new_artists (artist_id, Artist, followers, genres, popularity)
SELECT artist_id, Artist, followers, genres, popularity
FROM artists;

-- drop artists table
DROP TABLE artists;

-- rename new_artists to artists
ALTER TABLE new_artists RENAME TO artists;


   sqlite:///music_database.db
   sqlite:///music_database_2.db
 * sqlite:///rs_greatest_albums.db
Done.
Done.
387 rows affected.
Done.
Done.


[]

In [77]:
%%sql

    SELECT album_id
    FROM (
        SELECT album_id, ROW_NUMBER() OVER (PARTITION BY album_id ORDER BY album_id) AS row_num
        FROM albums
    ) AS subquery
    WHERE row_num > 1
;


   sqlite:///music_database.db
   sqlite:///music_database_2.db
 * sqlite:///rs_greatest_albums.db
Done.


album_id


In [78]:
%%sql
-- same basic idea for albums
DROP TABLE IF EXISTS new_albums;
CREATE TABLE new_albums(
    album_id STRING PRIMARY KEY,
    Album TEXT,
    artist_id STRING,
    Year INTEGER,
    rank_2003 INTEGER,
    rank_2012 INTEGER,
    rank_2020 INTEGER,
    rank_2023 INTEGER,
    FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
);

INSERT INTO new_albums
SELECT * FROM albums;

DROP TABLE albums;

ALTER TABLE new_albums RENAME TO albums;

   sqlite:///music_database.db
   sqlite:///music_database_2.db
 * sqlite:///rs_greatest_albums.db
Done.
Done.
634 rows affected.
Done.
Done.


[]

In [79]:
%%sql
-- Same idea for tracks
DROP TABLE IF EXISTS new_tracks;
CREATE TABLE new_tracks(
    track_id STRING PRIMARY KEY,
    track_name TEXT,
    album_id STRING,
    track_number INTEGER,
    duration_ms INTEGER,
    FOREIGN KEY (album_id) REFERENCES albums(album_id)
);

INSERT INTO new_tracks
SELECT * FROM tracks;
DROP TABLE tracks;
ALTER TABLE new_tracks RENAME TO tracks;

   sqlite:///music_database.db
   sqlite:///music_database_2.db
 * sqlite:///rs_greatest_albums.db
Done.
Done.
9780 rows affected.
Done.
Done.


[]

In [80]:
%%sql
-- count duplicate track_id
SELECT track_id, COUNT(*) AS count
FROM tracks
GROUP BY track_id
HAVING count > 1;

   sqlite:///music_database.db
   sqlite:///music_database_2.db
 * sqlite:///rs_greatest_albums.db
Done.


track_id,count


In [81]:
%%sql
-- select artists with > 5 ranked albums
SELECT artists.Artist, COUNT(DISTINCT albums.album_id) AS num_albums
FROM artists
JOIN albums ON artists.artist_id = albums.artist_id
GROUP BY artists.Artist
HAVING num_albums > 5;

   sqlite:///music_database.db
   sqlite:///music_database_2.db
 * sqlite:///rs_greatest_albums.db
Done.


Artist,num_albums
Beatles,10
"Bowie, David",7
"Dylan, Bob",11
"John, Elton",6
"Marley, Bob",6
Rolling Stones,10
"Springsteen, Bruce",9
"West, Kanye",6
"Who, The",7
